In [1]:
import os

# Your desired directory path
desired_path = "c:/Projects/taxi_demand_predictory/"

# Change the current working directory to the specified path
os.chdir(desired_path)

# Verify by printing the current working directory
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  c:\Projects\taxi_demand_predictory


In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29,1,0.0
1,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2023-01-30,1,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2023-01-31,1,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-01,1,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40540,14.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,...,4.0,8.0,5.0,4.0,11.0,3.0,8.0,2023-06-26,265,5.0
40541,2.0,1.0,2.0,4.0,1.0,0.0,2.0,2.0,0.0,2.0,...,8.0,3.0,9.0,5.0,3.0,5.0,7.0,2023-06-27,265,3.0
40542,2.0,3.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,...,8.0,6.0,2.0,7.0,7.0,8.0,3.0,2023-06-28,265,4.0
40543,0.0,5.0,0.0,2.0,0.0,1.0,1.0,3.0,5.0,7.0,...,15.0,11.0,3.0,2.0,1.0,8.0,7.0,2023-06-29,265,9.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(7950, 674)
y_test.shape=(7950,)


Baseline Model 1

In [5]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']

In [6]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0       0.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
7945    8.0
7946    7.0
7947    3.0
7948    7.0
7949    4.0
Name: rides_previous_1_hour, Length: 7950, dtype: float32

In [7]:
from sklearn.metrics import mean_absolute_error, r2_score

test_mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'{test_mae=:.4f}')
print(f'{r2=:.4f}')

test_mae=7.3489
r2=0.6733


Baseline Model 2

In [8]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_{7*24}_hour']

In [9]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [10]:
from sklearn.metrics import mean_absolute_error, r2_score

test_mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'{test_mae=:.4f}')
print(f'{r2=:.4f}')

test_mae=4.0439
r2=0.8667


Model Baseline 3

In [11]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        """"""
        return 0.25*(
            X_test[f'rides_previous_{7*24}_hour'] + \
            X_test[f'rides_previous_{2*7*24}_hour'] + \
            X_test[f'rides_previous_{3*7*24}_hour'] + \
            X_test[f'rides_previous_{4*7*24}_hour']
        )

In [12]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [13]:
from sklearn.metrics import mean_absolute_error, r2_score

test_mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'{test_mae=:.4f}')
print(f'{r2=:.4f}')

test_mae=2.9619
r2=0.9279
